In [229]:
import pandas as pd
import numpy as np

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

import keras
from keras.models import Sequential
from keras.layers import Dense

from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, classification_report

In [12]:
data = pd.read_csv('/home/maksonvinicio/Documents/GitHub/Data-Science-Projects/Stroke Prediction/data/healthcare-dataset-stroke-data.csv')

data = data.drop(['id'], axis=1)

In [13]:
data.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [219]:
X = data.drop(['stroke'], axis=1)
y = data['stroke']

# unique pra descobrir quantos rótulos únicos
cols = ['ever_married', 'Residence_type']
le = LabelEncoder()
X[cols] = X[cols].apply(le.fit_transform)

ct = ColumnTransformer(transformers=[('ohe', OneHotEncoder(), ['gender', 'work_type', 'smoking_status'])],
                       remainder='passthrough')
X = ct.fit_transform(X)


colunas = ['ohe1', 'ohe2', 'ohe3', 'ohe4', 'ohe5', 'ohe6', 'ohe7', 'ohe8', 'ohe9', 'ohe10', 'ohe11', 'ohe12',
           'age', 'hypertension', 'heart_disease', 'ever_married', 'Residence_type',
           'avg_glucose_level', 'bmi']

X = pd.DataFrame(X, columns=colunas)


X = X.astype(np.number)


X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.4, random_state=42)


In [220]:
from sklearn.preprocessing import StandardScaler
cols_scaler = ['age', 'avg_glucose_level', 'bmi']
sc = StandardScaler()
X_train[cols_scaler] = sc.fit_transform(X_train[cols_scaler])
X_val[cols_scaler] = sc.transform(X_val[cols_scaler])

/home/maksonvinicio/.local/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
/home/maksonvinicio/.local/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [221]:
X_train.isna().sum()

ohe1                   0
ohe2                   0
ohe3                   0
ohe4                   0
ohe5                   0
ohe6                   0
ohe7                   0
ohe8                   0
ohe9                   0
ohe10                  0
ohe11                  0
ohe12                  0
age                    0
hypertension           0
heart_disease          0
ever_married           0
Residence_type         0
avg_glucose_level      0
bmi                  117
dtype: int64

In [239]:
from sklearn.ensemble import ExtraTreesRegressor

estimator = ExtraTreesRegressor(n_estimators=10, random_state=42)
it_imp = IterativeImputer(estimator=estimator, missing_values=np.nan, max_iter=10)

X_train['bmi'] = it_imp.fit_transform(X_train['bmi'].values.reshape(-1, 1))
X_val['bmi'] = it_imp.fit_transform(X_val['bmi'].values.reshape(-1, 1))

/home/maksonvinicio/.local/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
/home/maksonvinicio/.local/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [243]:
y_val.value_counts()

0    1930
1     114
Name: stroke, dtype: int64

In [240]:
X_train

,ohe1,ohe2,ohe3,ohe4,ohe5,ohe6,ohe7,ohe8,ohe9,ohe10,ohe11,ohe12,age,hypertension,heart_disease,ever_married,Residence_type,avg_glucose_level,bmi
3563,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.929328,0.0,1.0,1.0,0.0,0.097954,0.651907
3245,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.595444,1.0,0.0,1.0,0.0,-0.918263,-0.250384
3966,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.669349,0.0,0.0,0.0,1.0,-0.690822,0.497228
2940,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.328998,1.0,1.0,1.0,1.0,-0.852513,0.510118
4824,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.684260,0.0,0.0,1.0,0.0,-0.305939,1.038603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4426,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.003233,0.0,0.0,1.0,0.0,-0.391369,0.226541
466,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.796105,1.0,0.0,1.0,0.0,1.443591,4.041943
3092,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,-1.861252,0.0,0.0,0.0,1.0,-0.183832,-1.423363
3772,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.639852,0.0,0.0,1.0,0.0,2.025723,0.278100


In [248]:
model = Sequential()
model.add(Dense(units=6, activation='relu', input_dim=19))
model.add(Dense(units = 6, activation = 'relu'))
model.add(Dense(units = 6, activation = 'relu'))
model.add(Dense(units = 1, activation = 'sigmoid'))

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.fit(X_train, y_train, batch_size = 10, epochs = 30)

Epoch 1/30
307/307 [==============================] - 1s 849us/step - loss: 0.4019 - accuracy: 0.9211
Epoch 2/30
307/307 [==============================] - 0s 827us/step - loss: 0.1672 - accuracy: 0.9560
Epoch 3/30
307/307 [==============================] - 0s 787us/step - loss: 0.1574 - accuracy: 0.9560
Epoch 4/30
307/307 [==============================] - 0s 820us/step - loss: 0.1518 - accuracy: 0.9560
Epoch 5/30
307/307 [==============================] - 0s 809us/step - loss: 0.1474 - accuracy: 0.9560
Epoch 6/30
307/307 [==============================] - 0s 812us/step - loss: 0.1461 - accuracy: 0.9560
Epoch 7/30
307/307 [==============================] - 0s 908us/step - loss: 0.1451 - accuracy: 0.9560
Epoch 8/30
307/307 [==============================] - 0s 939us/step - loss: 0.1442 - accuracy: 0.9560
Epoch 9/30
307/307 [==============================] - 0s 884us/step - loss: 0.1436 - accuracy: 0.9560
Epoch 10/30
307/307 [==============================] - 0s 1ms/step - loss: 0.1433 

In [249]:
pred = model.predict(X_val)
pred = (pred > 0.5)

# Resultados

In [250]:
print("Relatório de Classificação:\n", classification_report(y_val, pred, digits=4))

Relatório de Classificação:
               precision    recall  f1-score   support

           0     0.9442    1.0000    0.9713      1930
           1     0.0000    0.0000    0.0000       114

    accuracy                         0.9442      2044
   macro avg     0.4721    0.5000    0.4857      2044
weighted avg     0.8916    0.9442    0.9171      2044



/home/maksonvinicio/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [251]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_val, pred) 
#vn, fp(rede disse que é 1, mas é 0), fn (rede disse que é 0, mas é 1), vp

cm

array([[1930,    0],
       [ 114,    0]])

In [252]:
accuracy_score(y_val, pred)

0.9442270058708415